In [1]:
import pandas as pd
import geopandas as gpd
import time
import os
import warnings
from pandas import read_csv

warnings.filterwarnings("ignore")
os.chdir('/Users/xiaodanxu/Library/CloudStorage/GoogleDrive-arielinseu@gmail.com/My Drive/GEMS/BILD-AQ/data')
# selected_state = 'MS'

In [2]:
microtype_lookup = read_csv('ccst_geoid_key_tranps_geo_with_imputation.csv', sep = ',')
# state_list = microtype_lookup['st_code'].unique()
# state_list = ['AZ', 'NV']

In [ ]:
census_tract_shapefile = 'Network/combined/census_tracts_2017.geojson'
polygon_gdf = gpd.read_file(census_tract_shapefile)
polygon_gdf = polygon_gdf.to_crs('EPSG:4326')
# polygon_gdf.plot()
from pygris.utils import shift_geometry

polygon_gdf_rescaled = shift_geometry(polygon_gdf)
polygon_gdf_rescaled.plot()

In [4]:
# perform spatial intersection
for selected_state in state_list:
    if selected_state == 'OH':
        continue
    print('processing state ' + selected_state)
    output_dir = 'internal/' + selected_state
    route_list = [file for file in os.listdir(output_dir) if file.endswith('.geojson')]
#     print(route_list[0:5])
    start_time = time.time()
    for route in route_list:
        if route.startswith('._'):
            continue
        file_name = route.split('.geojson')[0]
        isExist = os.path.exists(output_dir + '/' + file_name + '.csv')
        if isExist:
            continue
        print('processing route ' + route)
        line_gdf = gpd.read_file(output_dir + '/' + route)
    #     sample_line_gdf = line_gdf.head(100)
        # print(start_time)
        line_by_polygon = gpd.overlay(line_gdf, polygon_gdf, how='intersection')

        # compute segment length in meters
        line_by_polygon = line_by_polygon.to_crs("EPSG:3310") 
        # in order to get length in meter, the shapefile need to re-projected to a coordinate system in meters (not required in R)
        line_by_polygon.loc[:, 'Length'] = line_by_polygon.loc[:, 'geometry'].length
        
        line_by_polygon_df = pd.DataFrame(line_by_polygon.drop(columns='geometry'))
        line_by_polygon_df = line_by_polygon_df[['source',	'destination',
                                                  'distance', 'GEOID', 'Length']]
    #     line_by_polygon_df.columns = []
    #     print(len(line_by_polygon))
        line_by_polygon_df.to_csv(output_dir + '/' + file_name + '.csv', index = False)

    #     break
    end_time = time.time()
    total_time = end_time - start_time
    print('spatial intersection in Python takes ' + str(total_time) + ' sec')

processing state AZ
processing route result-4-csv_0.geojson
processing route result-4-csv_1.geojson
processing route result-4-csv_2.geojson
processing route result-4-csv_3.geojson
processing route result-4-csv_4.geojson
processing route result-4-csv_5.geojson
processing route result-20-csv_0.geojson
processing route result-20-csv_1.geojson
processing route result-20-csv_2.geojson
processing route result-20-csv_3.geojson
processing route result-20-csv_4.geojson
processing route result-20-csv_5.geojson
spatial intersection in Python takes 14131.783043146133 sec
processing state NV
processing route result-5-csv-3_0.geojson
processing route result-5-csv-3_1.geojson
spatial intersection in Python takes 1981.9662718772888 sec


In [5]:
line_by_polygon_df.head(5)

,source,destination,distance,GEOID,Length
0,40143007407,40105172200,99326.245394,40143005900,10072.073880
1,40143007407,40105172300,83091.556274,40143005900,10072.073880
2,40143007407,40105172400,83826.376872,40143005900,10072.073880
3,40143007407,40113940002,109694.228375,40143005900,1325.930235
4,40143007407,40113940004,103867.863385,40143005900,1325.930235
